# Auto-Encoder for Iris Data

In [19]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from matplotlib.colors import ListedColormap

In [47]:
iris = datasets.load_iris()
X = iris.data
t = iris.target
target_names = iris.target_names

print('Size of the data = ', X.shape)
n = X.shape[0]
mdim = X.shape[1]
print('N=', n, 'mdim=', mdim)
print('size of t', t.shape)

# normarize X
xmean = np.mean(X, axis=0)
xsd = np.std(X, axis=0)
X = (X - xmean)/xsd
print(X)


Size of the data =  (150, 4)
N= 150 mdim= 4
size of t (150,)
[[-9.00681170e-01  1.01900435e+00 -1.34022653e+00 -1.31544430e+00]
 [-1.14301691e+00 -1.31979479e-01 -1.34022653e+00 -1.31544430e+00]
 [-1.38535265e+00  3.28414053e-01 -1.39706395e+00 -1.31544430e+00]
 [-1.50652052e+00  9.82172869e-02 -1.28338910e+00 -1.31544430e+00]
 [-1.02184904e+00  1.24920112e+00 -1.34022653e+00 -1.31544430e+00]
 [-5.37177559e-01  1.93979142e+00 -1.16971425e+00 -1.05217993e+00]
 [-1.50652052e+00  7.88807586e-01 -1.34022653e+00 -1.18381211e+00]
 [-1.02184904e+00  7.88807586e-01 -1.28338910e+00 -1.31544430e+00]
 [-1.74885626e+00 -3.62176246e-01 -1.34022653e+00 -1.31544430e+00]
 [-1.14301691e+00  9.82172869e-02 -1.28338910e+00 -1.44707648e+00]
 [-5.37177559e-01  1.47939788e+00 -1.28338910e+00 -1.31544430e+00]
 [-1.26418478e+00  7.88807586e-01 -1.22655167e+00 -1.31544430e+00]
 [-1.26418478e+00 -1.31979479e-01 -1.34022653e+00 -1.44707648e+00]
 [-1.87002413e+00 -1.31979479e-01 -1.51073881e+00 -1.44707648e+00]
 

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [48]:
X = torch.tensor(X, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

print(X.shape, t.shape)  # 出力: (150, 4) (150,)

print(type(X))

# データセットの分割割合
n_train = int(len(X) * 0.8)
n_test = len(X) - n_train

train, test = torch.utils.data.random_split(torch.utils.data.TensorDataset(X, t), [n_train, n_test])

# DataLoaderの定義
batch_size = 10
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test, batch_size)


torch.Size([150, 4]) torch.Size([150])
<class 'torch.Tensor'>


In [57]:
for i, (xdata, tdata) in enumerate(train_loader):
    print(i)
    print(xdata)
    print(tdata)

0
tensor([[-0.1737, -0.5924,  0.4217,  0.1325],
        [ 1.6438,  1.2492,  1.3311,  1.7121],
        [-1.0218,  0.7888, -1.2834, -1.3154],
        [ 1.1592, -0.5924,  0.5922,  0.2641],
        [-0.1737, -0.5924,  0.1944,  0.1325],
        [-0.9007, -1.2830, -0.4308, -0.1308],
        [-0.4160, -1.7434,  0.1375,  0.1325],
        [-1.2642, -0.1320, -1.3402, -1.4471],
        [-0.2948, -0.8226,  0.2512,  0.1325],
        [ 1.2803,  0.0982,  0.7628,  1.4488]])
tensor([1, 2, 0, 1, 1, 1, 1, 0, 1, 2])
1
tensor([[ 0.4322, -1.9736,  0.4217,  0.3958],
        [-1.3854,  0.3284, -1.2266, -1.3154],
        [ 1.2803,  0.0982,  0.9333,  1.1856],
        [-1.0218, -1.7434, -0.2603, -0.2624],
        [ 0.7957, -0.1320,  1.1606,  1.3172],
        [-0.2948, -0.3622, -0.0898,  0.1325],
        [-0.5372,  0.7888, -1.2834, -1.0522],
        [-1.5065,  1.2492, -1.5676, -1.3154],
        [-1.0218,  0.3284, -1.4539, -1.3154],
        [-0.4160, -1.5132,  0.0239, -0.1308]])
tensor([1, 0, 2, 1, 2, 1, 0, 0, 0, 

In [49]:
# Multi Layer Perceptron Network
n_hidden = 2
class AENet (nn.Module):
    def __init__(self):
        super(AENet, self).__init__()
        self.fc1 = nn.Linear(mdim, n_hidden)
#        self.bn1 = nn.BatchNorm1d(num_features=n_hidden, affine=True)
#        self.dropout1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(n_hidden, mdim)
        
    def forward(self, x):
        y = self.fc1(x)
#        y = self.bn1(y)
#        y = self.dropout1(y)
#        y = y + 0.1*torch.randn(n_hidden).cuda()
        z = self.fc2(y)
        return z

In [60]:
# select device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = AENet().to(device)
print(device)

# optimizing
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0000001)
#optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=0.001)

#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)


cpu


In [62]:
###  training
print ('training start ...')
num_epochs = 500   

#lrs = []

# initialize list for plot graph after training
train_loss_list = []

for epoch in range(num_epochs):
    # initialize each epoch
    train_loss = 0
    
    # ======== train_mode (Backprop) ======
    net.train()
    for i, (xdata, tdata) in enumerate(train_loader):  # get mini batch samples
#        print(i, xdata)
        xdata = xdata.to(device)
        optimizer.zero_grad()  # Reset the gradients
        outputs = net(xdata)  # forward computation
        loss = criterion(outputs, xdata)  # loss
        loss.backward()  # backward computation        
        optimizer.step()  # optimization

    # ======== update learning rate ======
#    lrs.append(optimizer.param_groups[0]["lr"])
#    print("Learning Rate = ",optimizer.param_groups[0]["lr"])
#    scheduler.step()

    
    # ======== eval_mode (training samples) ======
    net.eval()
    with torch.no_grad():  # no computation of gradients
      for (xdata, tdata) in train_loader:        
          xdata = xdata.to(device)
          outputs = net(xdata)
          loss = criterion(outputs, xdata)
          train_loss += loss.item()
#          acc = (outputs.max(1)[1] == labels).sum()
#          train_acc += acc.item()
    avg_train_loss = train_loss / len(train_loader.dataset)
#    avg_train_acc = train_acc / len(train_loader.dataset)
    
    # print log
    print ('Epoch [{}/{}] train Loss: {loss:.8f} ' 
                   .format(epoch+1, num_epochs, i+1, loss=avg_train_loss))

    # append list for polt graph after training
    train_loss_list.append(avg_train_loss)

# plt.plot(range(num_epochs), lrs)

training start ...
Epoch [1/500] train Loss: 0.00437227 
Epoch [2/500] train Loss: 0.00437133 
Epoch [3/500] train Loss: 0.00437054 
Epoch [4/500] train Loss: 0.00436959 
Epoch [5/500] train Loss: 0.00436874 
Epoch [6/500] train Loss: 0.00436797 
Epoch [7/500] train Loss: 0.00436708 
Epoch [8/500] train Loss: 0.00436640 
Epoch [9/500] train Loss: 0.00436556 
Epoch [10/500] train Loss: 0.00436462 
Epoch [11/500] train Loss: 0.00436382 
Epoch [12/500] train Loss: 0.00436309 
Epoch [13/500] train Loss: 0.00436231 
Epoch [14/500] train Loss: 0.00436149 
Epoch [15/500] train Loss: 0.00436069 
Epoch [16/500] train Loss: 0.00436003 
Epoch [17/500] train Loss: 0.00435925 
Epoch [18/500] train Loss: 0.00435856 
Epoch [19/500] train Loss: 0.00435773 
Epoch [20/500] train Loss: 0.00435702 
Epoch [21/500] train Loss: 0.00435631 
Epoch [22/500] train Loss: 0.00435556 
Epoch [23/500] train Loss: 0.00435482 
Epoch [24/500] train Loss: 0.00435415 
Epoch [25/500] train Loss: 0.00435341 
Epoch [26/500] 